In [1]:
###############################################
#           H E A D E R   F I L E S
###############################################
import os
import csv
import numpy as np
from Prj_XPS2CurveChart_01 import GetPaths, PlotCurve

###############################################
#          F U N C T I O N   L I S T
###############################################
## @brief Description: calculate the mean values of the csv files
#  @param [in] files : csv files
#  
#  @return mean values
#  @date 20230304  danielwu
def MeanValues(files):
    success = 0
    mean_values = np.zeros(800) # correct range is -200 to 599
    
    for file in files:
        # open csv file
        with open(file, newline='') as f:
            # read csv file
            rows = list(csv.reader(f))

            # if it is not correct range (-200 - 599), excluded the file
            if rows[0][0] != '-200' or rows[-1][0] != '599':
                basename = os.path.basename(file)
                print(f'[WRONG RANGE] {basename} is excluded!')
                continue

            # calculate the sum of the values
            for i, row in enumerate(rows):
                mean_values[i] += float(row[1])
        
        success += 1

    # calculate the mean values
    mean_values = np.divide(mean_values, success)

    return mean_values

## @brief Description: write the data into the csv file
#  @param [in] save_path : output csv file path
#  @param [in] x_coors   : data x coordinates
#  @param [in] y_coors   : data y coordinates
#  
#  @return None
#  @date 20230305  danielwu
def WriteCSV(save_path, x_coors, y_coors):
    # save pixels coors to csv
    with open(save_path, 'w', newline='') as f:
        writer = csv.writer(f) # make csv file writer
        writer.writerows(zip(x_coors, y_coors)) # write table in the csv file
###############################################
#             D A T A   T Y P E S
###############################################

###############################################
#              C O N S T A N T S
###############################################

###############################################
#        G L O B A L   V A R I A B L E
###############################################
# set root path
ROOT = 'C:\\Users\\danielwu\\Desktop\\ee\\xps_ALL\\xps_NHT'

###############################################
#                   M A I N
###############################################
if __name__ == '__main__':
    # get csv files
    files = GetPaths(ROOT, '.csv')
    
    # classified by frequency
    freq_4k_files = list(filter(lambda x: '4kHz' in x, files))
    freq_2k_files = list(filter(lambda x: '2kHz' in x, files))
    freq_1k_files = list(filter(lambda x: '1kHz' in x, files))
    freq_500_files = list(filter(lambda x: '500Hz' in x, files))
    
    # get each frequency mean values
    freq_4k_mean_values = MeanValues(freq_4k_files)
    freq_2k_mean_values = MeanValues(freq_2k_files)
    freq_1k_mean_values = MeanValues(freq_1k_files)
    freq_500_mean_values = MeanValues(freq_500_files)
    
    # combine basename and data (the data is y coordinate)
    outputs = {'out_4kHz.csv': freq_4k_mean_values,
               'out_2kHz.csv': freq_2k_mean_values,
               'out_1kHz.csv': freq_1k_mean_values,
               'out_500Hz.csv': freq_500_mean_values,}
    
    # set x coordinate
    x_coors = list(range(-200,600))
    
    # create output folder if it not exist
    output_dir = os.path.join(ROOT, 'output')
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
    
    for basename in outputs:
        # set output csv file path and save the csv file
        save_path = os.path.join(output_dir, basename)
        WriteCSV(save_path, x_coors, outputs[basename])
        
        # set path of output curve chart, plot it and save it
        freq = basename.split('.')[0]
        save_path = os.path.join(output_dir, f'{freq}.png')
        PlotCurve(x_coors, outputs[basename], freq, save_path)
    
    print('done')

done
